In [16]:
import os
import json
import requests
from bs4 import BeautifulSoup
from openai import OpenAI

configs = {
    "openai": {
        "key": "OPENAI_API_KEY",
        "url": "https://api.openai.com/v1",
        "model": "gpt-4o-mini",
    },
    "groq": {
        "key": "GROQ_API_KEY",
        "url": "https://api.groq.com/openai/v1",
        "model": "llama-3.1-8b-instant",
    },
    "ollama": {
        "key": "",
        "url": "https://localhost:11434/api/v1",
        "model": "llama3.1",
    },
}

In [18]:
# Google search
SEARCH_SCHEMA = {
    "type": "function",
    "function": {
        "name": "search",
        "description": "Perform a Google search given a query string.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The search query to be used.",
                },
            },
            "required": ["query"],
        },
    },
}


SCRAPE_SCHEMA = {
    "type": "function",
    "function": {
        "name": "scrape",
        "description": "Extract and return the text content from the webpage at the given URL. Only the first 2000 characters of paragraph text are returned.",
        "parameters": {
            "type": "object",
            "properties": {
                "url": {
                    "type": "string",
                    "description": "The URL of the webpage to scrape for text content.",
                },
            },
            "required": ["url"],
        },
    },
}


def scrape(url: str) -> str:
    try:
        # Send an HTTP GET request to the URL
        response = requests.get(url)

        # Check if the request was successful (status code 200)
        if response.status_code != 200:
            return ""

        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, "html.parser")

        # Extract the text content of the webpage
        texts = "\n".join([p.get_text().strip() for p in soup.find_all("p")])[:2000]

        return texts
    except Exception as e:
        print(f"An error occurred: {e}")
        return ""


def parse_search_item(search_item: dict[str, any]) -> dict[str, str]:
    try:
        long_description = search_item["pagemap"]["metatags"][0]["og:description"]
    except KeyError:
        long_description = "N/A"
    title = search_item.get("title")
    description = search_item.get("snippet")
    url = search_item.get("link")
    return {
        "title": title,
        "description": description,
        "long_description": long_description,
        "url": url,
    }


def search(query: str) -> list[dict[str, str]]:
    try:
        results = []
        params = {
            "key": os.getenv("GOOGLE_SEARCH_API_KEY"),
            "cx": os.getenv("GOOGLE_SEARCH_ENGINE_ID"),
            "q": query,
            "start": 1,
        }
        # Make the HTTP GET request to the Custom Search JSON API
        response = requests.get(
            "https://www.googleapis.com/customsearch/v1", params=params
        )
        response.raise_for_status()  # Raise an exception for bad responses
        response = response.json()

        # Extract search results
        search_results = response.get("items", [])
        results.extend(
            [parse_search_item(search_item) for search_item in search_results[:3]]
        )
        return results

    except Exception as e:
        print(f"Error making Google search: {e}")
        return []


tool_schemas = [SEARCH_SCHEMA, SCRAPE_SCHEMA]
tools = {"search": search, "scrape": scrape}

In [25]:
config = configs["openai"]

client = OpenAI(
    api_key=os.getenv(config["key"]),
    base_url=config["url"],
)

results = [
    "Analyzing Kasper's vision - Inside The Vatican",
    "Raffaello Sanzio's (Raphael) - Marriage of the Virgin. | Art day ...",
    "Pinacoteca di Brera (Mailand) - Aktuelle 2019 - Lohnt es sich? (Mit fotos)",
    "Moja Top-lista, czyli drugi spacer po Pinacoteca di Brera (post_71 ...",
    "「ブレラ絵画館(ブレラ美術館)」完全ガイド – チケット予約・見どころ・行き方・混みぐあい",
    "Pinacoteca di Brera (Milan, Italy): Top Tips Before You Go (with Photos ...",
    "IMG_0228 The Marriage of the Virgin (1504), painting by Ra… | Flickr",
    "A master visual catechist: Raphael and the Italian High Renaissance ...",
    "Marriage of the Virgin after Raphael 'Raffaello Sanzio da Urbino', 1483 ...",
    "The Circumcision | Marco Marzile, 1500, oil on canvas. | Brule Laker ...",
]

results = "\n".join([f"{i + 1}. {r}" for i, r in enumerate(results)])

messages = [
    {
        "role": "system",
        "content": (
            "You are an esteemed art history professor with expertise in "
            "identifying artworks and interpreting their content. When "
            "presented with a list of visual search results, identify the "
            "actual artwork, including its title and artist. Then, describe "
            "the specific event or moment depicted in the piece. Use the "
            "tools if necessary to gather additional information to ensure "
            "your response is accurate and complete."
        ),
    },
    {
        "role": "user",
        "content": (
            f"Here is a list of visual search results of an artwork:\n\n{results}\n\n"
            f"Identify the true artwork, then describe the specific event "
            f"or moment depicted in that artwork."
        ),
    },
]
stop = False

while not stop:
    print(stop)
    completion = client.chat.completions.create(
        model=config["model"],
        messages=messages,
        tools=tool_schemas,
        stream=True,
    )

    output = ""
    tool_calls = {}
    for chunk in completion:
        if chunk.choices[0].finish_reason == "stop":
            messages.append({"role": "assistant", "content": output})
            stop = True

        elif chunk.choices[0].finish_reason == "tool_calls":
            messages.append(
                {
                    "role": "assistant",
                    "tool_calls": list(tool_calls.values()),
                }
            )

            for tool_call in tool_calls.values():
                tool = tool_call["function"]
                arguments = json.loads(tool["arguments"])
                result = tools[tool["name"]](**arguments)
                messages.append(
                    {
                        "role": "tool",
                        "tool_call_id": tool_call["id"],
                        "content": json.dumps({"result": result}),
                    }
                )

            tool_calls = {}

        else:
            delta = chunk.choices[0].delta

            if delta.tool_calls != None:
                for tool_call in delta.tool_calls:
                    if not tool_call.index in tool_calls:
                        tool_calls[tool_call.index] = {
                            "id": tool_call.id,
                            "type": "function",
                            "function": {
                                "name": tool_call.function.name,
                                "arguments": tool_call.function.arguments,
                            },
                        }
                    else:
                        tool_calls[tool_call.index]["function"][
                            "arguments"
                        ] += tool_call.function.arguments

            else:
                print(delta)
                output += delta.content

False
False
False
ChoiceDelta(content='', function_call=None, refusal=None, role='assistant', tool_calls=None)
ChoiceDelta(content='The', function_call=None, refusal=None, role=None, tool_calls=None)
ChoiceDelta(content=' artwork', function_call=None, refusal=None, role=None, tool_calls=None)
ChoiceDelta(content=' identified', function_call=None, refusal=None, role=None, tool_calls=None)
ChoiceDelta(content=' is', function_call=None, refusal=None, role=None, tool_calls=None)
ChoiceDelta(content=' **', function_call=None, refusal=None, role=None, tool_calls=None)
ChoiceDelta(content='"', function_call=None, refusal=None, role=None, tool_calls=None)
ChoiceDelta(content='Marriage', function_call=None, refusal=None, role=None, tool_calls=None)
ChoiceDelta(content=' of', function_call=None, refusal=None, role=None, tool_calls=None)
ChoiceDelta(content=' the', function_call=None, refusal=None, role=None, tool_calls=None)
ChoiceDelta(content=' Virgin', function_call=None, refusal=None, role=N

In [27]:
print(messages[-1]["content"])

The artwork identified is **"Marriage of the Virgin"** by **Raphael** (also known as Raffaello Sanzio), completed in **1504**. This oil painting is currently housed in the Pinacoteca di Brera in Milan, Italy.

### Description of the Event Depicted
The painting illustrates a significant moment in Christian tradition—the marriage ceremony between Mary and Joseph. In the artwork, we see Mary and Joseph standing before a priest as part of the wedding ritual, with a crowd of witnesses present. Mary is depicted as a serene figure adorned in a blue robe, symbolizing her purity, while Joseph, who stands on her right, shows a softer demeanor. 

The composition is characterized by its harmonious arrangement of figures and the architectural elements in the background, which are influenced by Renaissance ideals of balance and perspective. The use of vibrant colors and meticulous detail further showcases Raphael's mastery of the period. This piece not only emphasizes the sanctity of the marriage bu